In [2]:
!pip install tekore

In [3]:
!pip install python-dotenv

In [4]:
!pip install tqdm colorama
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [5]:
import tekore as tk
import os
from dotenv import load_dotenv
import pandas as pd
from tqdm import tqdm

In [6]:
# fuck you, load_dotenv
# you know what, load_dotenv? We got off on the wrong foot. I wanted you to be a certain thing, but you work in a way that's all your own I guess. I must let you be the free spirit you are.
load_dotenv()
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
client_id, client_secret

('340bdfadbf1b48d684111081b0be204d', '7d00cad90c844b7eb04e32e48711e372')

In [8]:
# Client id and client secret are the bare minimum for getting any API response.
# You can do more things relating to a user's history or playback settings once you have their OAuth verification token
# There's some added complexity to getting/storing that, so we're starting simple
app_token = tk.request_client_token(client_id, client_secret)
spotify = tk.Spotify(app_token)
spotify_ch = tk.Spotify(app_token,chunked_on=True)

In [9]:
all_cats_current = spotify.categories(limit=50)
for cat in all_cats_current.items:
    print(cat.name)
# These are all 48 categories 
# Thing I just realized: the order of these shifts depending on the time you retrieve them
# we're fixing this by hard-coding a master list of name, cat_id pairings so that the labels don't change during training
all_cats_master =  [['toplists', 'Top Lists'],
                    ['hiphop', 'Hip-Hop'],
                    ['summer', 'Summer'],
                    ['pop', 'Pop'],
                    ['country', 'Country'],
                    ['workout', 'Workout'],
                    ['latin', 'Latin'],
                    ['mood', 'Mood'],
                    ['rock', 'Rock'],
                    ['rnb', 'R&B'],
                    ['blackhistorymonth', 'Black History Is Now'],
                    ['gaming', 'Gaming'],
                    ['focus', 'Focus'],
                    ['edm_dance', 'Dance/Electronic'],
                    ['chill', 'Chill'],
                    ['at_home', 'At Home'],
                    ['indie_alt', 'Indie'],
                    ['inspirational', 'Christian'],
                    ['decades', 'Decades'],
                    ['alternative', 'Alternative'],
                    ['wellness', 'Wellness'],
                    ['pride', 'Pride'],
                    ['party', 'Party'],
                    ['sleep', 'Sleep'],
                    ['classical', 'Classical'],
                    ['jazz', 'Jazz'],
                    ['roots', 'Folk & Acoustic'],
                    ['soul', 'Soul'],
                    ['dinner', 'Cooking & Dining'],
                    ['romance', 'Romance'],
                    ['kpop', 'K-Pop'],
                    ['punk', 'Punk'],
                    ['regionalmexican', 'Regional Mexican'],
                    ['sessions', 'Spotify Singles'],
                    ['popculture', 'Pop culture'],
                    ['arab', 'Arab'],
                    ['desi', 'Desi'],
                    ['anime', 'Anime'],
                    ['thirdparty', 'Tastemakers'],
                    ['afro', 'Afro'],
                    ['comedy_v2_podcast', 'Comedy'],
                    ['metal', 'Metal'],
                    ['reggae', 'Reggae'],
                    ['blues', 'Blues'],
                    ['funk', 'Funk'],
                    ['student', 'Student'],
                    ['travel', 'Commute'],
                    ['family', 'Kids & Family']]

Top Lists
Hip-Hop
Summer
Pop
Country
Workout
Latin
Mood
Rock
R&B
Black History Is Now
Gaming
Focus
Dance/Electronic
Chill
At Home
Indie
Christian
Decades
Alternative
Wellness
Pride
Party
Sleep
Classical
Jazz
Folk & Acoustic
Soul
Cooking & Dining
Romance
K-Pop
Punk
Regional Mexican
Spotify Singles
Pop culture
Arab
Desi
Anime
Tastemakers
Afro
Comedy
Metal
Reggae
Blues
Funk
Student
Commute
Kids & Family


3 things:
- begin listing processing steps for each song
- script to compile all tracks under a category
- search genres, date range, duration range

### I'm starting with a look at one category
Many playlists can be tagged with one category label, so out of anticipation that dance/electronic is going to be pretty likely to sound the same, let's get a track list for the first playlist under the "Dance/Electronic" label

In [ ]:
edm_dance_list = spotify.category_playlists('edm_dance',limit=20)
edm_dance_list, edm_dance_list.items
# This is the object representation of a particular list of playlists, ".items" is most useful here

In [ ]:
mint = edm_dance_list.items[0]
mint # Not guaranteed to be "mint" playlist because the category order shifted above

### Let's get the first 2 songs' track data

In [ ]:
print("This just shows us the link to the playlist and how many songs are in it")
print(mint.tracks)
print("---\n","playlist_items is the class that gets us track ID's")
mint_tracks = spotify.playlist_items(mint.id).items
mint_tracks

In [ ]:
song1 = mint_tracks[0].track
song2 = mint_tracks[1].track
song1

In [ ]:
# audio_analysis and audio_features
song1_analysis = spotify.track_audio_analysis(song1.id)
song2_analysis = spotify.track_audio_analysis(song2.id)
song1_features = spotify.track_audio_features(song1.id)
song2_features = spotify.track_audio_features(song2.id)
song1_analysis, song1_features

### Combine all the track's data and compare correlation

In [ ]:
song1song2_compare = pd.DataFrame([[vars(song1), vars(song1_features)],
                                   [vars(song2), vars(song1_features)]])
song1song2_compare.head()

# New activity: Condense an entire mood category

In [ ]:
track_analysis = spotify.track_audio_analysis(track.track.id)
track_features = spotify.track_audio_features(track.track.id)


In [ ]:
# for section in track_analysis.sections:
#     print(f'Key:{section.key} Conf:{section.key_confidence}')
track_analysis.
# everything needs expanding
# don't need bars, beats, meta

In [10]:
import numpy as np
import time

alb_cols = ['album_lead_artist','album_name','release_date','total_tracks'] # these don't map exactly to the names they have in the object but it's what I want the DF columns to be
art_cols = ['track_artists','genres', 'artist_pop']
keep_cols = ['id','duration_ms','explicit','name','popularity','track_number']

feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'speechiness', 'tempo', 'time_signature', 'valence']
df_colnames = ['cat_idx'] + alb_cols + art_cols + keep_cols + feature_cols
#question marks: episode/track, disc_number, restrictions, track_number
#album question marks: album_group, album_type

In [11]:
def basic_track_data(full_track, full_playlist_track=False):
    '''
    Using object type FullPlaylistTrack, this function returns a row of data with columns matching what the final DataFrame of training set is going to look like.
    A track of this type can be retrieved by calling (with Tekore): 
            paging_obj = spotify.playlist_items(playlist_placeholder.id) #playlist paging object
            paging_obj.items[track_index] #returns FullPlaylistTrack object type
    
    We're not using any playlist-specific fields here, ~~~"so it should work the same with a regular FullTrack object"~~~ this is wrong.
    It does not, so the default input is now FullTrack object, and if full_playlist_track is True, it converts
    '''
    if full_playlist_track:
        full_track = full_track.track
    # expand album field, retrieve album data labeled by alb_expand_cols
    alb_data = []
    alb_data.append(full_track.album.artists[0].name)
    alb_data.append(full_track.album.name)
    alb_data.append(full_track.album.release_date)
    alb_data.append(full_track.album.total_tracks)
    
    # expand artists field, list all artists as string separated by newlines
    artists_str = ''
    for artist in full_track.artists:
        artists_str = artists_str + artist.name + '\n'
    
    # lookup lead artist, list genres as string, separated by newlines
    genres_str = ''
    lead_artist = spotify.artist(full_track.artists[0].id)
    for genre in lead_artist.genres:
        genres_str = genres_str + genre + '\n'
    # record lead artist popularity and save
    artist_pop = lead_artist.popularity
    art_data = [artists_str, genres_str, artist_pop]
    
    # generate data as labeled by keep_cols
    track_data = [full_track.__dict__[col] for col in keep_cols]
    
    # return single row containing all track data
    return alb_data + art_data + track_data

def audio_data(full_playlist_track): # don't use this 
    features_obj = spotify.track_audio_features(full_playlist_track.track.id)
    #analysis_obj = spotify.track_audio_analysis(full_playlist_track.track.id)
    
    #Will this be too many API calls? we'll find out. (it is possible to batch audio_features request but not audio_analysis)
    features_data = [features_obj.__dict__[fcol] for fcol in feature_cols]
    
    #Skipping analysis for now
    return features_data #+ analysis_data

#[df_colnames, *[basic_track_data(track)+ audio_data(track)]]


In [14]:
#training_songs = pd.DataFrame([], columns=df_colnames)
training_songs = pd.read_csv('training_data.csv') #Use this loading if adding to existing version of training_songs
sleep_min = 2
sleep_max = 5
start_time = time.time()
request_count = 0
#spotify = tk.Spotify(chunked_on=True)
with spotify_ch.chunked():
    # Loop thru categories here
    for i,cat in tqdm(enumerate(all_cats_master[11:]), desc='categories', leave=False):#currently picking up from index 11
        i=i+11
        # Loop thru playlists/subcategories here
        cat_playlists = spotify.category_playlists(cat[0],limit=10)
        cat_playlist_total = min(10, cat_playlists.total)
        #print(f'Accessing {cat_playlist_total} playlists')
        for p in tqdm(cat_playlists.items, total=cat_playlist_total, desc='playlists', leave=False):
            # Loop thru and collect tracks
            cat_playlist_tracks_pg = spotify.playlist_items(p.id) #could return "as_tracks"
            track_total = cat_playlist_tracks_pg.total
            cat_playlist_track_ids = [track.track.id if track.track!=None else None for track in spotify.all_items(cat_playlist_tracks_pg)]
            
            #Remove None's from ids list
            try:
                while cat_playlist_track_ids.index(None):
                    cat_playlist_track_ids.pop(cat_playlist_track_ids.index(None))
            except:
                pass
            
            cat_playlist_tracks = spotify_ch.tracks(cat_playlist_track_ids)
            cat_playlist_tracks_feat = spotify_ch.tracks_audio_features(cat_playlist_track_ids)
            #cat_playlist_tracks.__dir__()
            #cat_playlist_tracks.__sizeof__() # this returns 48 instead of 300, not sure why
            #print(f'   Collecting {track_total} songs in playlist')
            for j,track in enumerate(cat_playlist_tracks): # iterating like this reliably counts all 300
                request_count+=1
                if request_count%5==0:
                    time.sleep(np.random.uniform(sleep_min, sleep_max))
#                 if track.track == None:
#                     continue
                features_data = [cat_playlist_tracks_feat[j].__dict__[fcol] for fcol in feature_cols]
                training_songs = training_songs.append(dict(zip(df_colnames,[i]+basic_track_data(track)+features_data)), ignore_index=True)


categories: 0it [00:00, ?it/s]
playlists: 100%|█████████████████████████████████████████████████████████████████████████| 6/6 [05:24<00:00, 53.81s/it]
categories: 1it [05:24, 324.99s/it]                                                                                    
playlists: 100%|█████████████████████████████████████████████████████████████████████████| 7/7 [16:27<00:00, 96.42s/it]
categories: 2it [21:52, 523.67s/it]                                                                                    
playlists:  70%|█████████████████████████████████████████████████▋                     | 7/10 [12:45<05:29, 109.95s/it]


ServiceUnavailable: Error in https://api.spotify.com/v1/artists/5HydVvU98cVWguCYzh7WbA:
503: Service unavailable


In [16]:
training_songs.shape
training_songs['cat_idx'].max()
# for i,cat in enumerate(all_cats_master[11:]):
#     print(i+11,cat)

11 ['gaming', 'Gaming']
12 ['focus', 'Focus']
13 ['edm_dance', 'Dance/Electronic']
14 ['chill', 'Chill']
15 ['at_home', 'At Home']
16 ['indie_alt', 'Indie']
17 ['inspirational', 'Christian']
18 ['decades', 'Decades']
19 ['alternative', 'Alternative']
20 ['wellness', 'Wellness']
21 ['pride', 'Pride']
22 ['party', 'Party']
23 ['sleep', 'Sleep']
24 ['classical', 'Classical']
25 ['jazz', 'Jazz']
26 ['roots', 'Folk & Acoustic']
27 ['soul', 'Soul']
28 ['dinner', 'Cooking & Dining']
29 ['romance', 'Romance']
30 ['kpop', 'K-Pop']
31 ['punk', 'Punk']
32 ['regionalmexican', 'Regional Mexican']
33 ['sessions', 'Spotify Singles']
34 ['popculture', 'Pop culture']
35 ['arab', 'Arab']
36 ['desi', 'Desi']
37 ['anime', 'Anime']
38 ['thirdparty', 'Tastemakers']
39 ['afro', 'Afro']
40 ['comedy_v2_podcast', 'Comedy']
41 ['metal', 'Metal']
42 ['reggae', 'Reggae']
43 ['blues', 'Blues']
44 ['funk', 'Funk']
45 ['student', 'Student']
46 ['travel', 'Commute']
47 ['family', 'Kids & Family']


In [196]:
#ts_backup = training_songs #---uncomment to store current version
#training_songs = ts_backup #---uncomment to reset DF to last backup
# for i in range(training_songs['cat_idx'].max()+2):
#     print(i,(training_songs['cat_idx']==i).sum())
i

NameError: name 'ids' is not defined

In [109]:
training_songs.to_csv('training_data.csv')

# New New activity: make weird searches

In [19]:
spotify.search('popularity:54',types='track')

BadRequest: Error in https://api.spotify.com/v1/search?q=popularity%3A54&type=t%2Cr%2Ca%2Cc%2Ck&limit=20&offset=0:
400: Bad search type field t


In [20]:
same_as_song1 = spotify.track('2dh6Pnl5egc1FrQS6EsW4n')
# this is a cool thing that I don't actually need: track_keys = [*same_as_song1.__dict__.keys()]
for key, value in same_as_song1.__dict__.items():
    try:
        result = spotify.search(f'{key}:{value}',types=('track',),limit=20)
        if result[0].total>0: print(result)
    except:
        pass
    else:
        print('^---',key,', ',value,'---^')

^--- id ,  2dh6Pnl5egc1FrQS6EsW4n ---^
^--- href ,  https://api.spotify.com/v1/tracks/2dh6Pnl5egc1FrQS6EsW4n ---^
(FullTrackPaging with fields:
  href = 'https://api.spotify.com/v1/search?query=type%3Atrack&type=tra...'
  items = [20 x FullTrack(album, artists, available_markets, ...)]
  limit = 20
  next = 'https://api.spotify.com/v1/search?query=type%3Atrack&type=tra...'
  offset = 0
  previous = None
  total = 283,)
^--- type ,  track ---^
^--- uri ,  spotify:track:2dh6Pnl5egc1FrQS6EsW4n ---^
^--- artists ,  ModelList with items: [
  SimpleArtist(external_urls, href, id, name, type, uri)
  SimpleArtist(external_urls, href, id, name, type, uri)
] ---^
(FullTrackPaging with fields:
  href = 'https://api.spotify.com/v1/search?query=disc_number%3A1&type=...'
  items = [20 x FullTrack(album, artists, available_markets, ...)]
  limit = 20
  next = 'https://api.spotify.com/v1/search?query=disc_number%3A1&type=...'
  offset = 0
  previous = None
  total = 99,)
^--- disc_number ,  1 ---^
^--

Does searching for multiple artists the goofy way actually return results?

In [27]:
spotify.search('genre:"rock"', types=('tracks',), limit=20)

BadRequest: Error in https://api.spotify.com/v1/search?q=genre%3A%22rock%22&type=tracks&limit=20&offset=0:
400: Bad search type field tracks


In [107]:
for hot_explicits in spotify.search('explicit:True')[0].items:
    name = hot_explicits.name
    artist1 = hot_explicits.artists[0].name
    pop = hot_explicits.popularity
    #year = hot_explicits.year      can't do year?
    print(name+' by '+artist1)
    print('Popularity: ',pop,'\n---')
    #print('Released ',year)

I'm Throwed (feat. Jermaine Dupri) by Paul Wall
Popularity:  47 
---
Break Em' Off (feat. Lil' Keke) by Paul Wall
Popularity:  44 
---
Bangin Screw by Paul Wall
Popularity:  31 
---
Everybody Know Me (feat. Snoop Dogg) by Paul Wall
Popularity:  24 
---
I Ain't Hard to Find by Paul Wall
Popularity:  22 
---
Gimme That by Paul Wall
Popularity:  23 
---
Get Your Paper Up (feat. Yung Redd) by Paul Wall
Popularity:  19 
---
Tonight (feat. Jon B) by Paul Wall
Popularity:  18 
---
Call Me What U Want (feat. Yung Redd & E Class) by Paul Wall
Popularity:  18 
---
That Fire (feat. Trina) by Paul Wall
Popularity:  19 
---
Break Em' Off (feat. Lil' Keke) - C&S Version by Paul Wall
Popularity:  17 
---
How Gangstas Roll (feat. Crys Wall) by Paul Wall
Popularity:  16 
---
On the Grind (feat. Freeway & Crys Wall) by Paul Wall
Popularity:  15 
---
I'm Real, What Are You? (feat. Juelz Santana) by Paul Wall
Popularity:  14 
---
Slidin' on That Oil (feat. Expensive Taste) by Paul Wall
Popularity:  12 
--

In [87]:
for hipster in spotify.search('tag:hipster')[0].items:
    name = hipster.name
    artist1 = hipster.artists[0].name
    print(name+' by '+artist1)
    print('Popularity: ',pop,'\n---')

Valse mélancolique in F-Sharp Minor, Op. 332 "Le regret" (Previously attributed to Chopin) by Charles Mayer
Popularity:  5 
---
Holz, Holz geigt die Quartette so (Attrib. Beethoven as WoO 204) by Karl Holz
Popularity:  5 
---
Soft pink noise putting down baby by Colic Remedies Pink Noise
Popularity:  5 
---
ASMR deep sleep by Easy Falling Asleep ASMR
Popularity:  5 
---
Calm down fussy baby by Colic Remedies Pink Noise
Popularity:  5 
---
Better night shusher by Pink Noise for Better Night
Popularity:  5 
---
Healing ASMR sounds by Easy Falling Asleep ASMR
Popularity:  5 
---
Brown noise airplane sound by Brown Noise Enjoyable Sleep
Popularity:  5 
---
Baby stops crying by Brown Noise Enjoyable Sleep
Popularity:  5 
---
Fall asleep easily by Easy Falling Asleep ASMR
Popularity:  5 
---
White noise sleep aid all night by Soothing Airplane Sound All Night
Popularity:  5 
---
Soft colic remedies by Colic Remedies Pink Noise
Popularity:  5 
---
Airplane sound chill out by Soothing Airplane

# I need to chunk

In [136]:
spotify_ch = tk.Spotify(app_token,chunked_on=True)
#spotify_ch.chunked_on = False

with spotify_ch.chunked():
    tracks_chunk = spotify_ch.tracks(list(training_songs['id'][:100]))
    tracks_chunk_feat = spotify_ch.tracks_audio_features(list(training_songs['id'][:100]))
#spotify.tracks(list(training_songs['id'][:100]))

In [148]:
# cat_playlist_track_ids = [track.track.id for track in spotify.all_items(cat_playlist_tracks)]
# [track.track.id for track in spotify.all_items(spotify.playlist_items(p.id))]
for track in tracks_chunk:
    basic_track_data(track)

AttributeError: 'FullTrack' object has no attribute 'track'